<a href="https://colab.research.google.com/github/aoonging/dataviz2025/blob/main/midterm2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

#อ่านไฟล์

In [ ]:
import pandas as pd

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd

directory_path = '/content/drive/MyDrive/Exam Data DataViz2025/'
file_list = os.listdir(directory_path)

for file_name in file_list:
    if file_name.endswith('.csv'):
        file_path = os.path.join(directory_path, file_name)
        try:
            df = pd.read_csv(file_path)
            print(f"Columns in {file_name}: {df.columns.tolist()}")
        except Exception as e:
            print(f"Error reading {file_name}: {e}")

Columns in get_fit_now_check_in.csv: ['membership_id', 'check_in_date', 'check_in_time', 'check_out_time']
Columns in drivers_license.csv: ['id', 'age', 'height', 'eye_color', 'hair_color', 'gender', 'plate_number', 'car_make', 'car_model']
Columns in interview.csv: ['person_id', 'transcript']
Columns in facebook_event_checkin.csv: ['person_id', 'event_id', 'event_name', 'date']
Columns in person.csv: ['id', 'name', 'license_id', 'address_number', 'address_street_name', 'ssn']
Columns in income.csv: ['ssn', 'annual_income']
Columns in get_fit_now_member.csv: ['id', 'person_id', 'name', 'membership_start_date', 'membership_status']
Columns in crime_scene_report.csv: ['date', 'type', 'description', 'city']


In [ ]:
# โหลดไฟล์
person = pd.read_csv('/content/drive/MyDrive/Exam Data DataViz2025/person.csv')
members = pd.read_csv("/content/drive/MyDrive/Exam Data DataViz2025/get_fit_now_member.csv")
checkins = pd.read_csv("/content/drive/MyDrive/Exam Data DataViz2025/get_fit_now_check_in.csv")
drivers = pd.read_csv("/content/drive/MyDrive/Exam Data DataViz2025/drivers_license.csv")
fb = pd.read_csv("/content/drive/MyDrive/Exam Data DataViz2025/facebook_event_checkin.csv")
income = pd.read_csv("/content/drive/MyDrive/Exam Data DataViz2025/income.csv")
crimedfre = pd.read_csv('/content/drive/MyDrive/Exam Data DataViz2025/crime_scene_report.csv')
interviews_df = pd.read_csv('/content/drive/MyDrive/Exam Data DataViz2025/interview.csv')

#สืบหาฆาตรกร และผู้บงการ

In [ ]:
#ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder)
#ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv
murder_report = crimedfre[
    (crimedfre['date'] == 20180115) &
    (crimedfre['type'] == 'murder') &
    (crimedfre['city'] == 'Pandas City')
]

print(murder_report['description'].iloc[0])

Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


จากภาพกล้องวงจรปิด พบว่ามีพยาน 2 คน

พยานคนแรก อาศัยอยู่ที่ บ้านหลังสุดท้ายบนถนน Northwestern Dr

พยานคนที่สอง ชื่อ Annabel อาศัยอยู่ที่ใดที่หนึ่งบนถนน Franklin Ave

Now I will check the `interviews` data to find the statements of the two witnesses.

In [ ]:
#หาชื่อพยานคนที่1
# Find the first witness: last house on "Northwestern Dr"
northwestern_residents = person[person['address_street_name'] == 'Northwestern Dr']
last_house_northwestern = northwestern_residents[northwestern_residents['address_number'] == northwestern_residents['address_number'].max()]

print("First Witness (last house on Northwestern Dr):")
display(last_house_northwestern)


#หาชื่อพยานคนที่2
# ชื่อ Annabel
annabel_candidates = person[person['name'].str.contains("Annabel", case=False, na=False)]

# หาคนที่อยู่บน Franklin Ave
franklin_residents = person[person['address_street_name'].str.contains("Franklin", case=False, na=False)]

# หาคนที่เป็น Annabel + อยู่บน Franklin Ave
annabel_franklin = annabel_candidates.merge(franklin_residents, how="inner")

print("Second Witness (Annabel on Franklin Ave):")
display(annabel_franklin)

First Witness (last house on Northwestern Dr):


,id,name,license_id,address_number,address_street_name,ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


Second Witness (Annabel on Franklin Ave):


,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [ ]:
# พยานที่เรามี id
witness_ids = [14887, 16371]

# ดึง transcript
witness_transcripts = interviews_df[interviews_df['person_id'].isin(witness_ids)].merge(
    person[['id','name']], left_on='person_id', right_on='id', how='left'
)[['person_id','name','transcript']]

print(witness_transcripts.to_string(index=False))

 person_id           name                                                                                                                                                                                                                      transcript
     14887 Morty Schapiro I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".
     16371 Annabel Miller                                                                                                           I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


**transcript พยาน**

Morty Schapiro:
“ฉันได้ยินเสียงปืนดังขึ้น แล้วก็เห็นผู้ชายคนหนึ่งวิ่งออกมา เขามีกระเป๋าของ ‘Get Fit Now Gym’ หมายเลขสมาชิกบนกระเป๋าเริ่มด้วย ‘48Z’ มีแค่สมาชิกระดับทองเท่านั้นที่มีกระเป๋าแบบนี้ ผู้ชายคนนั้นขึ้นรถที่ป้ายทะเบียนมีตัวอักษร ‘H42W’”

Annabel Miller:
“ฉันเห็นเหตุการณ์ฆาตกรรมเกิดขึ้น และฉันจำคนร้ายได้จากยิมของฉัน ตอนที่ฉันกำลังออกกำลังกายเมื่อสัปดาห์ที่แล้ว วันที่ 9 มกราคม”

In [ ]:
import pandas as pd

# กรอง Gold member และ membership_id ที่เริ่มด้วย 48Z
gold_members = members[
    (members['membership_status'].str.lower() == 'gold') &
    (members['id'].astype(str).str.startswith('48Z'))
]

gold_members


,id,person_id,name,membership_start_date,membership_status
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


In [ ]:
# Find the person with a car plate including "H42W"
suspect_license = drivers[drivers['plate_number'].str.contains('H42W', case=False, na=False)]

print("Suspect based on car plate:")
display(suspect_license)

Suspect based on car plate:


,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
915,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
3529,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
6240,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [ ]:
#Merge เพื่อดูว่าใครเป็นเจ้าของรถแต่ละคัน รวมข้อมูลรถของผู้ต้องสงสัยกับข้อมูลบุคคล
suspect_info = suspect_car.merge(person, left_on='id', right_on='license_id', how='left')
#เลือกเฉพาะคอลัมน์สำคัญที่ต้องการดู
suspect_info[['name', 'address_number', 'address_street_name', 'ssn']]


,name,address_number,address_street_name,ssn
0,Maxine Whitely,110,Fisk Rd,137882671
1,Jeremy Bowers,530,"Washington Pl, Apt 3A",871539279
2,Tushar Chandra,312,Phi St,137882671


In [ ]:
gold_checkins = gold_members.merge(checkins, left_on='id', right_on='membership_id', how='left')

# เลือก check-in ในวันเกิดเหตุ
gold_checkins_on_event = gold_checkins[gold_checkins['check_in_date'] == 20180109] # Corrected date based on interview

gold_checkins_on_event

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,48Z7A,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700


In [ ]:
suspect_mastermind = suspect_info.merge(income, on='ssn', how='left')
suspect_mastermind[['name', 'annual_income']]


,name,annual_income
0,Maxine Whitely,NaN
1,Jeremy Bowers,10500.0
2,Tushar Chandra,NaN


In [ ]:
import pandas as pd
import re

# โหลดไฟล์ interview.csv
interview = pd.read_csv("/content/drive/MyDrive/Exam Data DataViz2025/interview.csv")

# กำหนด pattern ที่สื่อถึงการจ้างหรือสั่งฆาตรกร
mastermind_keywords = [r'hired', r'paid', r'ordered']

# ฟังก์ชันค้นหา keyword ใน transcript
def find_mastermind_keywords(text):
    found = []
    for kw in mastermind_keywords:
        if re.search(kw, str(text), flags=re.IGNORECASE):
            found.append(kw)
    return found

# สร้างคอลัมน์ clues จากการค้นหา keyword
interview['mastermind_clues'] = interview['transcript'].apply(find_mastermind_keywords)

# กรองเฉพาะคนที่เจอ clue → ผู้บงการ
suspected_mastermind = interview[interview['mastermind_clues'].map(len) > 0]

# แสดงชื่อและ clue
print(suspected_mastermind[['person_id', 'transcript', 'mastermind_clues']])


      person_id                                         transcript  \
1895      96376      number of executions the Queen had ordered.\n   
1914      76405  after it: ‘I never was so ordered about in all...   
2695      92977      number of executions the Queen had ordered.\n   
4246      52579  executions I have ordered’; and she walked off...   
4990      67318  I was hired by a woman with a lot of money. I ...   

     mastermind_clues  
1895        [ordered]  
1914        [ordered]  
2695        [ordered]  
4246        [ordered]  
4990          [hired]  


In [ ]:

# กรองข้อมูล Maxine Whitely จาก person
maxine_info = person[person['name'] == 'Maxine Whitely']

# รวมกับ income เพื่อดูรายได้
maxine_info_income = maxine_info.merge(income, on='ssn', how='left')

# แสดงผล
print(maxine_info_income)


      id            name  license_id  address_number address_street_name  \
0  78193  Maxine Whitely      183779             110             Fisk Rd   

         ssn  annual_income  
0  137882671            NaN  


#สรุปตัวฆาตกรและผู้บงการ

เหตุผลในการระบุฆาตรกรและผู้บงการ
1. ฆาตรกร: Jeremy Bowers

Transcript ของพยาน Morty Schapiro ระบุว่า ผู้ร้ายถือ กระเป๋าของ “Get Fit Now Gym” และเป็น สมาชิกระดับทอง (Gold member)

หมายเลขสมาชิกของกระเป๋าเริ่มด้วย "48Z" → เมื่อเปรียบเทียบกับฐานข้อมูลสมาชิก พบว่า Jeremy Bowers ตรงกับ Gold membership “48Z55”

Transcript ระบุว่าผู้ร้ายขึ้น รถทะเบียน H42W → เชื่อมกับฐานข้อมูลรถ พบว่า Jeremy Bowers เป็นเจ้าของรถที่ใกล้เคียงกับ clue

ตรวจสอบ check-in ของยิมในวันที่เกิดเหตุ พบว่า Jeremy ไม่มีการลงเวลา → ไม่มี alibi

Transcript ของ Annabel Miller ยืนยันว่าเห็น Jeremy ที่ยิมก่อนวันเกิดเหตุ → confirm clue และยืนยันความเป็นสมาชิก Gold

person_id 67318  I was hired by a woman with a lot of money. I ...   ซึ่งคือ Jeremy ที่ถูกจ้างวานจากผู้หญิงที่รวย

ฆาตรกรคือ Jeremy Bowers → Transcript ระบุว่าเขา "was hired"

ดังนั้น Jeremy Bowers จึงตรงกับ ทุก clue ของฆาตรกร

  2. สรุปผู้บงการ (Maxine Whitely)

Transcript ของ Jeremy Bowers ระบุว่าเขา "was hired by a woman with a lot of money" → แสดงว่ามีผู้หญิงจ้างฆาตรกร

จากฐานข้อมูลบุคคล (person.csv) คนเดียวที่เป็นผู้หญิงและเกี่ยวข้องกับคดีคือ Maxine Whitely

ข้อมูลเพศจาก drivers_license.csv ยืนยันว่า Maxine เป็นผู้หญิง → ตรงกับ clue

Merge กับรถที่ฆาตรกรขึ้น พบว่า ถ้าไม่ใช่รถของ Jeremy เอง Maxine อาจจะเป็นเจ้าของรถทะเบียนที่ Jeremy ใช้  → สนับสนุนบทบาทผู้บงการ